In [ ]:
%pip install seaborn
%pip install plotly
%pip install nbformat

In [ ]:
import numpy as np

import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model as lm
import plotly.express as px
import nbformat

import warnings
warnings.filterwarnings("ignore")

import zipfile
import os

import hashlib

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

In [ ]:
initial_data = pd.read_csv("cook_county_train.csv", index_col='Unnamed: 0')

# Exploratory Data Analysis

In [ ]:
# 204792 observations and 62 features in training data
assert initial_data.shape == (204792, 62)
# Sale Price is provided in the training data
assert 'Sale Price' in initial_data.columns.values

In [ ]:
initial_data.columns.values

In [ ]:
initial_data['Description'][0]

View of the distribution, a limited range of house prices from [$0, $1,000,000].

In [ ]:
no_right_outliers = initial_data[initial_data['Sale Price'] <= 1000000]['Sale Price']
plt.hist(no_right_outliers, bins=200)
plt.xlabel("Sale Price")
plt.ylabel("Count")
plt.title("Distribution of Sale Price")
plt.show()

The proportion of how many buildings sold for over \\$1,000,000.

In [ ]:
len(initial_data[initial_data['Sale Price'] > 1000000]) / len(initial_data[initial_data['Sale Price'] >= 500])

In [ ]:
training_data = initial_data[initial_data['Sale Price'] >= 500].copy()
training_data['Log Sale Price'] = np.log(training_data['Sale Price'])

In [ ]:
def plot_distribution(data, label):
    fig, axs = plt.subplots(nrows=2)

    sns.distplot(
        data[label], 
        ax=axs[0]
    )
    sns.boxplot(
        x=data[label],
        width=0.3, 
        ax=axs[1],
        showfliers=False,
    )

    # Align axes
    spacer = np.max(data[label]) * 0.05
    xmin = np.min(data[label]) - spacer
    xmax = np.max(data[label]) + spacer
    axs[0].set_xlim((xmin, xmax))
    axs[1].set_xlim((xmin, xmax))

    # Remove some axis text
    axs[0].xaxis.set_visible(False)
    axs[0].yaxis.set_visible(False)
    axs[1].yaxis.set_visible(False)

    # Put the two plots together
    plt.subplots_adjust(hspace=0)
    fig.suptitle("Distribution of " + label)

In [ ]:
plot_distribution(training_data, label='Log Sale Price');

In [ ]:
def remove_outliers(data, variable, lower=-np.inf, upper=np.inf):
    filtered_data = data[(data[variable] > lower) & (data[variable] < upper)].copy()
    return filtered_data

Check for duplicates

In [ ]:
tr_val_data = pd.read_csv("cook_county_train_val.csv", index_col='Unnamed: 0')
test_data = pd.read_csv("cook_county_contest_test.csv", index_col='Unnamed: 0')

In [ ]:
duplicates = tr_val_data[tr_val_data.duplicated(keep=False)]
duplicate_counts = duplicates.groupby(duplicates.columns.tolist()).size()
count_duplicate_properties = (duplicate_counts > 1).sum()

count_duplicate_rows_to_remove = duplicate_counts.sum() - count_duplicate_properties

print("There are ", count_duplicate_properties, "unique property sales with exact duplicates.")
print("There are ", count_duplicate_rows_to_remove, "a total of duplicate rows that we'll need to remove when we write our cleaning function below.")

In [ ]:
missing_values = tr_val_data['Sale Price'].isnull().sum() 
missing_values += tr_val_data['Sale Price'].isna().sum()
negative_or_zero_values = (tr_val_data['Sale Price'] <= 0).sum()
print(missing_values + negative_or_zero_values)

Q1 = tr_val_data['Sale Price'].quantile(0.25)
Q3 = tr_val_data['Sale Price'].quantile(0.75)
IQR = Q3 - Q1

upper_limit = Q3 + 1.5 * IQR
large_outliers = (tr_val_data['Sale Price'] >= upper_limit).sum()
print(large_outliers)

lower_limit = Q1 + 1.5 * IQR
small_outliers = (tr_val_data['Sale Price'] <= lower_limit).sum()
print(small_outliers)

In [ ]:
pure_market_sales = tr_val_data[tr_val_data['Pure Market Filter'] == 1]
max_Sale_Price_filtered = pure_market_sales['Sale Price'].max()

min_Sale_Price_filtered = pure_market_sales['Sale Price'].min()

print("When considering only pure market sales, the max Sale Price of properties in the data is $", max_Sale_Price_filtered)
print("and the min Sale Price is $", min_Sale_Price_filtered)

In [ ]:

def clean_data(data):
    da = data.copy()
    da = da.drop_duplicates()
    da = da[da['Pure Market Filter'] == 1]    
    return da

# Feature Engineering

In [ ]:
def add_total_bedrooms(data):
    with_rooms = data.copy()

    bedrooms_regex = r'(\d+) of which are bedrooms'
    
    with_rooms['Bedrooms'] = with_rooms['Description'].str.extract(bedrooms_regex).astype(float).fillna(0).astype(int)
    
    
    return with_rooms

training_data = add_total_bedrooms(training_data)

Find if there is an association between Bedrooms and Log Sale Price

In [ ]:
sns.boxplot(x='Bedrooms', y='Log Sale Price', data=training_data)

plt.xlabel('Number of Bedrooms')
plt.ylabel('Log Sale Price')
plt.title('Distribution of Log Sale Price by Number of Bedrooms')

Now looking at the relationship between neighborhood and sale prices of the houses

In [ ]:
num_neighborhoods = training_data['Neighborhood Code'].nunique()
num_neighborhoods

In [ ]:
neighborhood_sales_count = training_data['Neighborhood Code'].value_counts()
top_20 = neighborhood_sales_count.head(20).index

in_top_20_neighborhoods = training_data[training_data['Neighborhood Code'].isin(top_20)].copy()

In [ ]:
def plot_categorical(neighborhoods):
    fig, axs = plt.subplots(nrows=2)

    sns.boxplot(
        x='Neighborhood Code',
        y='Log Sale Price',
        data=neighborhoods,
        ax=axs[0],
    )

    sns.countplot(
        x='Neighborhood Code',
        data=neighborhoods,
        ax=axs[1],
    )

    # Draw median price
    axs[0].axhline(
        y=training_data['Log Sale Price'].median(), 
        color='red',
        linestyle='dotted'
    )

    # Label the bars with counts
    for patch in axs[1].patches:
        x = patch.get_bbox().get_points()[:, 0]
        y = patch.get_bbox().get_points()[1, 1]
        axs[1].annotate(f'{int(y)}', (x.mean(), y), ha='center', va='bottom')

    # Format x-axes
    axs[1].set_xticklabels(axs[1].xaxis.get_majorticklabels(), rotation=90)
    axs[0].xaxis.set_visible(False)

    # Narrow the gap between the plots
    plt.subplots_adjust(hspace=0.01)

In [ ]:
plot_categorical(neighborhoods=in_top_20_neighborhoods)

In [ ]:
def find_expensive_neighborhoods(data, n=3, metric=np.median):
    """
    Output:
      a list of the the neighborhood codes of the top n highest-priced neighborhoods as measured by the metric function
    """
    neighborhoods = data.groupby('Neighborhood Code')['Log Sale Price'].apply(metric)
    neighborhoods = neighborhoods.sort_values(ascending=False).head(n).index
    
    return [int(code) for code in neighborhoods]
    # return list(neighborhoods.index)
    
expensive_neighborhoods = find_expensive_neighborhoods(training_data, 3, np.median)
expensive_neighborhoods

In [ ]:
def add_in_expensive_neighborhood(data, expensive_neighborhoods):

    data['in_expensive_neighborhood'] = data['Neighborhood Code'].isin(expensive_neighborhoods).astype(int)
    return data

expensive_neighborhoods = find_expensive_neighborhoods(training_data, 3, np.median)
training_data = add_in_expensive_neighborhood(training_data, expensive_neighborhoods)



Looking into Roof Material

In [ ]:
def substitute_roof_material(data):
    roof_material_mapping = {
        1: 'Shingle/Asphalt',
        2: 'Tar&Gravel',
        3: 'Slate',
        4: 'Shake', 
        5: 'Tile',
        6: 'Other'
    }
    
    data_mapped = data.copy()
    data_mapped['Roof Material'] = data_mapped['Roof Material'].replace(roof_material_mapping)
    return data_mapped
    
training_data_mapped = substitute_roof_material(training_data)
training_data_mapped.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

In [ ]:
def ohe_roof_material(data):
    ohe = OneHotEncoder()
    ohe.fit(data[['Roof Material']])
    ohe.get_feature_names_out()
    ohe.transform(data[['Roof Material']])
    
    roof_material_encoded = ohe.transform(data[['Roof Material']]).todense()
    
    roof_material_df = pd.DataFrame(roof_material_encoded, columns=ohe.get_feature_names_out(), index=data.index)
    
    data_with_ohe = data.copy()
    data_with_ohe = data_with_ohe.join(roof_material_df)
    
    return data_with_ohe
training_data_ohe = ohe_roof_material(training_data_mapped)
training_data_ohe.filter(regex='^Roof Material').head(5)

# Cross Validation

In [ ]:
def train_val_split(data):
    """ 
    Takes in a DataFrame `data` and splits it into two smaller DataFrames 
    named `validation` and `train` where validation is the first 20% of the rows and train 
    is the last 80% of the rows, respectively. 
    """
    da = data.copy()
    
    cutoff = int(len(da) * 0.2)
    
    validation = da[:cutoff]
    train = da[cutoff:]
    
    return train, validation


# Randomize the validation and training sets
tr_val_data_shuffled = tr_val_data.sample(frac=1, random_state=18)

# Clean the shuffled data
tr_val_clean = clean_data(tr_val_data_shuffled) 

# Create the train/val split on the cleaned, shuffled data:
tr, val = train_val_split(tr_val_clean)

# Fitting a Simple Linear Regression Model

In [ ]:


def process_data_m1(df):
    """ 
    Takes in a DataFrame of cleaned data and performs feature engineering to use for Model 1.

    Outputs a DataFrame with only the features and response/output used in model 1 (that is `Log Sale Price` , `Log Building Square Feet`)
 
    """
    
    data=df.copy()
    
    # Add a column "Log Sale Price" to the `data` DataFrame:

    data['Log Sale Price'] = np.log(data['Sale Price'])
    
    # Add a column "Log Building Square Feet" to the `data` DataFrame:

    data['Log Building Square Feet'] = np.log(data['Building Square Feet'])
    
    # Select the feature and the output/response used in model 1:
    
    data = data[['Log Building Square Feet', 'Log Sale Price']]
    
    return data



# Process both the training and validation data: 
processed_train_m1 = process_data_m1(tr)

processed_val_m1 = process_data_m1(val)


# Create X (dataframe) and Y (series) to use to train the model:
X_train_m1 = processed_train_m1.drop(columns = "Log Sale Price")
y_train_m1 = processed_train_m1["Log Sale Price"]


# Create X (dataframe) and Y (series) to use to validate the model:
X_valid_m1 = processed_val_m1.drop(columns = "Log Sale Price")
y_valid_m1 = processed_val_m1["Log Sale Price"]

# Take a look at the results
print("Training Data: X")
display(X_train_m1.head())
print("Training Data: y")
display(y_train_m1.head())


print("Validation Data: X")
display(X_valid_m1.head())
print("Validation Data: y")
display(y_valid_m1.head())


In [ ]:
linear_model_m1 = lm.LinearRegression()

# Fit the model using the processed training data:

linear_model_m1.fit(X_train_m1, y_train_m1)

# Compute the predicted y values from linear model 1 (in units log sale price) 
# using the training data as input:

y_predict_train_m1 = linear_model_m1.predict(X_train_m1)

# Compute the predicted y values from linear model 1 (in units log(sale price))
# using the validation data as input:

y_predict_valid_m1 = linear_model_m1.predict(X_valid_m1)

In [ ]:
def rmse(predicted, actual):
    # Calculates RMSE from actual and predicted values
    
    mean_squared_diff = np.mean((actual - predicted) ** 2)
    return np.sqrt(mean_squared_diff)
    

In [ ]:
model_names=["M1: log(bsqft)", "M2", "M3"]

# Create arrays where we can keep track of training and validation RMSE for each model

training_error_log = np.zeros(4)
validation_error_log = np.zeros(4)

training_error = np.zeros(4)
validation_error = np.zeros(4)

# Array to track cross validation errors average RMSE errors  

cv_error = np.zeros(4)


In [ ]:
# Training and validation RMSE for the model (in units log sale price)

training_error_log[0] = rmse(y_train_m1, y_predict_train_m1)
validation_error_log[0]= rmse(y_valid_m1, y_predict_valid_m1)


# Training and validation RMSE for the model (in its original dollar values before the log transform)

training_error[0] = rmse(np.exp(y_train_m1), np.exp(y_predict_train_m1))
validation_error[0] = rmse(np.exp(y_valid_m1), np.exp(y_predict_valid_m1))

print("1st Model \nTraining RMSE: $ {}\nValidation RMSE: $ {}\n".format(training_error[0], validation_error[0]))

In [ ]:
# Cross Validation
from sklearn.model_selection import KFold

kf = KFold(n_splits=5) 

i = 1

a = []

for train_idx, valid_idx in kf.split(tr_val_clean):
    print ("positional (iloc) indices for training data for fold", i)
    print (train_idx)
    print ("positional (iloc) indices for validation data for fold", i)
    print (valid_idx)
    i = i+1

In [ ]:
from sklearn.model_selection import KFold
from sklearn.base import clone

def cross_validate_rmse(model, X, y):
    '''
    Split the X and y data into 5 subsets.
    For each subset, 
        - Fit a model holding out that subset.
        - Compute the RMSE (in units dollars, not log(dollars) on that subset (the validation set).
    Return the average RMSE of these 5 folds.
    '''
    # Make a copy of the model to use in this function
    model = clone(model)

    # Initialize sklearn's KFold object 
    kf = KFold(n_splits=5)  

    # Create a list to store the validation_rmse for each fold
    validation_rmse = []
    
    for train_idx, valid_idx in kf.split(X):

        split_X_train, split_X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        split_Y_train, split_Y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        # Fit the model on the training split:
        model.fit(split_X_train, split_Y_train)
        
        # Compute the RMSE (in units dollars, not log(dollars)) on the validation split:
        y_predict_valid = model.predict(split_X_valid)
        error = rmse(np.exp(split_Y_valid), np.exp(y_predict_valid))
    

        validation_rmse.append(error)
        

        #Return the average validation rmse across all cross-validation splits.

    cv_error = np.mean(validation_rmse)
              
        
    return cv_error
       
    
# Create a new model to use for cross validation of m1 
linear_model_m1_cv = lm.LinearRegression()


processed_full_m1 = process_data_m1(tr_val_clean)

X_full_m1 = processed_full_m1.drop(columns='Log Sale Price')
y_full_m1 = processed_full_m1['Log Sale Price']

cv_error_m1  = cross_validate_rmse(linear_model_m1_cv, X_full_m1, y_full_m1)

cv_error[0] = cv_error_m1

print("1st Model Cross Validation RMSE: {}".format(cv_error[0]))

# Visualizing RMSE

In [ ]:
import plotly.graph_objects as go

fig = go.Figure([
go.Bar(x = model_names, y = training_error, name="Training RMSE"),
go.Bar(x = model_names, y = validation_error, name="Validation RMSE"),
go.Bar(x = model_names, y = cv_error, name="Cross Val RMSE")
])

fig.update_yaxes(range=[180000,260000], title="RMSE")
fig

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15, 5))

residuals = y_valid_m1 - y_predict_valid_m1

x_plt1 = y_predict_valid_m1
y_plt1 = residuals

x_plt2 = y_valid_m1
y_plt2 = residuals



ax[0].scatter(x_plt1, y_plt1, alpha=.25)
ax[0].axhline(0, c='black', linewidth=1)
ax[0].set_xlabel(r'Predicted Log(Sale Price)')
ax[0].set_ylabel(r'Residuals: Log(Sale Price) - Predicted Log(Sale Price)');
ax[0].set_title("Model 1 Val Data: Residuals vs. Predicted Log(Sale Price)")

ax[1].scatter(x_plt2, y_plt2, alpha=.25)
ax[1].axhline(0, c='black', linewidth=1)
ax[1].set_xlabel(r'Log(Sale Price)')
ax[1].set_ylabel(r'Residuals: Log(Sale Price) - Predicted Log(Sale Price)');
ax[1].set_title("Model 1 Val Data: Residuals vs. Log(Sale Price)")

# Adding More Predictors 

In [ ]:
def process_data_candidates(df):
    
    data = df.copy()
    
    data["Log Sale Price"] = np.log(data["Sale Price"])
    
    # Create Log Building Square Feet column
    data["Log Building Square Feet"] = np.log(data["Building Square Feet"])
    
    
    # Create Bedrooms
    data = add_total_bedrooms(data)
     
   
    # Update Roof Material feature with names
    data = substitute_roof_material(data)

    
    # Select columns for comparing residuals
    data = data[['Log Building Square Feet',  'Roof Material', 'Bedrooms', 'Log Sale Price']]

    return data

    
valid_comp = process_data_candidates(val)
    
valid_comp = valid_comp.assign(M1residuals_log=y_valid_m1 - y_predict_valid_m1)


In [ ]:
import plotly.express as px

px.box(valid_comp, x='Bedrooms', y='M1residuals_log')

In [ ]:
px.box(valid_comp, x='Roof Material', y='M1residuals_log')

# Multiple Linear Regression Model

In [ ]:
#Process the Data
def substitute_roof_material(data):
    replacements = {
        'Roof Material': {
            1: 'Shingle/Asphalt',
            2: 'Tar&Gravel',
            3: 'Slate',
            4: 'Shake',
            5: 'Tile',
            6: 'Other',
        }
    }

    data = data.replace(replacements)
    return data

def process_data_m2(df):

    data = df.copy()

    data['Log Sale Price'] = np.log(data['Sale Price'])
    data['Log Building Square Feet'] = np.log(data['Building Square Feet'])
    
    data = substitute_roof_material(data)

    
    data = data[['Log Sale Price', 'Log Building Square Feet', 'Roof Material']]
    data = ohe_roof_material(data)
    data = data.drop(columns=['Roof Material'])
    
    
    return data

    
# Process the data for Model 2
processed_train_m2 = process_data_m2(tr)

processed_val_m2 = process_data_m2(val)


# Create X (dataframe) and Y (series) to use in the model
X_train_m2 = processed_train_m2.drop(columns = 'Log Sale Price')
y_train_m2 = processed_train_m2['Log Sale Price']

X_valid_m2 = processed_val_m2.drop(columns = 'Log Sale Price')
y_valid_m2 = processed_val_m2['Log Sale Price']


# Take a look at the result
display(X_train_m2.head())
display(y_train_m2.head())

display(X_valid_m2.head())
display(y_valid_m2.head())


In [ ]:
#Create and Fit a Multiple Linear Regression Model

processed_train_m2 = process_data_m2(tr)
processed_val_m2 = process_data_m2(val)

X_train_m2 = processed_train_m2.drop(columns = 'Log Sale Price')
y_train_m2 = processed_train_m2['Log Sale Price']

X_valid_m2 = processed_val_m2.drop(columns = 'Log Sale Price')
y_valid_m2 = processed_val_m2['Log Sale Price']

linear_model_m2 = lm.LinearRegression()
linear_model_m2.fit(X_train_m2, y_train_m2)

y_predict_train_m2 = linear_model_m2.predict(X_train_m2)

y_predict_valid_m2 = linear_model_m2.predict(X_valid_m2)




In [ ]:
# Evaluate the RMSE for the model

from sklearn.metrics import mean_squared_error
y_predict_train_m2_original = np.exp(y_predict_train_m2)
y_predict_valid_m2_original = np.exp(y_predict_valid_m2)

training_rmse = np.sqrt(mean_squared_error(np.exp(y_train_m2), y_predict_train_m2_original))
validation_rmse = np.sqrt(mean_squared_error(np.exp(y_valid_m2), y_predict_valid_m2_original))


training_error[1] = training_rmse
validation_error[1] = validation_rmse


print("2nd Model \nTraining RMSE: $ {}\nValidation RMSE: $ {}\n".format(training_error[1], validation_error[1]))


In [ ]:
# Conduct 5-fold cross validation for model and output CV RMSE

# Create a new model to use for cross validation of m2 
linear_model_m2_cv = lm.LinearRegression()

# Process the entire cleaned training_val dataset using the m2 pipeline
processed_full_m2 = process_data_m2(tr_val_clean)

# Split the processed_full_m2 Dataset into X and Y to use in models.
X_full_m2 = processed_full_m2.drop(columns='Log Sale Price')
y_full_m2 = processed_full_m2['Log Sale Price']


# Run cross_validate_rmse function:
cv_error_m2  = cross_validate_rmse(linear_model_m2_cv, X_full_m2, y_full_m2)

# Save the cross validation error for model 1 in our list to compare different models:

cv_error[1] = cv_error_m2

print("2nd Model Cross Validation RMSE: {}".format(cv_error[1]))







In [ ]:
# Plot bar graph comparing RMSEs of Model 2 and Model 1 and side-by-side residuals

model_names[1] = "M2: log(bsqft)+Roof"

fig = go.Figure([
go.Bar(x = model_names, y = training_error, name="Training RMSE"),
go.Bar(x = model_names, y = validation_error, name="Validation RMSE"),
go.Bar(x = model_names, y = cv_error, name="Cross Val RMSE")
])

fig.update_yaxes(range=[180000,260000], title="RMSE")

fig


In [ ]:
# Plot 2 side-by-side residual plots for validation data

fig, ax = plt.subplots(1,2, figsize=(15, 5))


x_plt1 = linear_model_m2.predict(X_valid_m2)
y_plt1 = y_valid_m2 - linear_model_m2.predict(X_valid_m2)

x_plt2 = y_valid_m2
y_plt2 = y_valid_m2 - linear_model_m2.predict(X_valid_m2)


ax[0].scatter(x_plt1, y_plt1, alpha=.25)
ax[0].axhline(0, c='black', linewidth=1)
ax[0].set_xlabel(r'Predicted Log(Sale Price)')
ax[0].set_ylabel(r'Residuals: Log(Sale Price) - Predicted Log(Sale Price)');
ax[0].set_title("Model 2 Val Data: Residuals vs. Predicted Log(Sale Price)")

ax[1].scatter(x_plt2, y_plt2, alpha=.25) 
ax[1].axhline(0, c='black', linewidth=1)
ax[1].set_xlabel(r'Log(Sale Price)')
ax[1].set_ylabel(r'Residuals: Log(Sale Price) - Predicted Log(Sale Price)');
ax[1].set_title("Model 2 Val Data: Residuals vs. Log(Sale Price)")


# Improving the Model

In [ ]:
# Choosing features to add
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
sns.scatterplot(x=tr_val_clean['Building Square Feet'], y=tr_val_clean['Sale Price'])
plt.title("Building Square Feet vs Sale Price")
plt.xscale('log')
plt.yscale('log')

plt.subplot(2, 2, 2)
sns.scatterplot(x=tr_val_clean['Land Square Feet'], y=tr_val_clean['Sale Price'])
plt.title("Land Square Feet vs Sale Price")
plt.xscale('log')
plt.yscale('log')

plt.subplot(2, 2, 3)
sns.scatterplot(x=tr_val_clean['Age'], y=tr_val_clean['Sale Price'])
plt.title("Age vs Sale Price")
plt.xscale('log')
plt.yscale('log')

plt.subplot(2, 2, 4)
sns.scatterplot(x=tr_val_clean['Fireplaces'], y=tr_val_clean['Sale Price'])
plt.title("Fireplaces vs Sale Price")
plt.xscale('log')
plt.yscale('log')

plt.tight_layout()
plt.show()
# Show work in this cell exploring data to determine which feature to add

In [ ]:
# Process the Data

def process_data_m3(df):
    
    data = df.copy()
        
    data['Log Sale Price'] = np.log(data['Sale Price'])
    data['Log Building Square Feet'] = np.log(data['Building Square Feet'])
    data['Log Land Square Feet'] = np.log(data['Land Square Feet'])
    
    
    data = substitute_roof_material(data)
    
    data = data[['Log Sale Price', 'Roof Material', 'Log Land Square Feet', 
                 'Log Building Square Feet']]
    data = ohe_roof_material(data) 

    data = data.drop(columns=['Roof Material'])
    
    
    return data

processed_train_m3 = process_data_m3(tr) 

processed_val_m3 = process_data_m3(val) 

# Create X (Dataframe) and y (series) to use to train the model
X_train_m3 = processed_train_m3.drop(columns= 'Log Sale Price')
y_train_m3 = processed_train_m3['Log Sale Price']

X_valid_m3 = processed_val_m3.drop(columns= 'Log Sale Price')
y_valid_m3 = processed_val_m3['Log Sale Price']


# Take a look at the result
display(X_train_m3.head())
display(y_train_m3.head())

display(X_valid_m3.head())
display(y_valid_m3.head())


In [ ]:
# Create and Fit a Multiple Linear Regression Model
processed_train_m3 = process_data_m3(tr)
processed_val_m3 = process_data_m3(val)

X_train_m3 = processed_train_m3.drop(columns = 'Log Sale Price')
y_train_m3 = processed_train_m3['Log Sale Price']

X_valid_m2 = processed_val_m3.drop(columns = 'Log Sale Price')
y_valid_m3 = processed_val_m3['Log Sale Price']

linear_model_m3 = lm.LinearRegression()
linear_model_m3.fit(X_train_m3, y_train_m3)

y_predict_train_m3 = linear_model_m3.predict(X_train_m3)

y_predict_valid_m3 = linear_model_m3.predict(X_valid_m3)



In [ ]:
# Evaluate the RMSE for your model

y_predict_train_m3_original = np.exp(y_predict_train_m3)
y_predict_valid_m3_original = np.exp(y_predict_valid_m3)

training_rmse = np.sqrt(mean_squared_error(np.exp(y_train_m3), y_predict_train_m3_original))
validation_rmse = np.sqrt(mean_squared_error(np.exp(y_valid_m3), y_predict_valid_m3_original))


training_error[2] = training_rmse
validation_error[2] = validation_rmse


(print("3rd Model \nTraining RMSE: $ {}\nValidation RMSE: {}\n"
       .format(training_error[2], validation_error[2]))
)


In [ ]:

# Conduct 5-fold cross validation for model and output RMSE

linear_model_m3_cv = lm.LinearRegression()

processed_full_m3 = process_data_m3(tr_val_clean)

# Split the processed_full_m3 Dataset into X and y to use in models.
X_full_m3 = processed_full_m3.drop(columns='Log Sale Price')
y_full_m3 = processed_full_m2['Log Sale Price']


# Run cross_validate_rmse function:
cv_error_m3  = cross_validate_rmse(linear_model_m3_cv, X_full_m3, y_full_m3)

# Save the cross validation error for model 3 in our list to compare different models:

cv_error[2] = cv_error_m3

print("3rd Model Cross Validation RMSE: {}".format(cv_error[2]))



In [ ]:
# Plot bar graph all 3 models

model_names[2] = "M3: log(lsqft)+log(bsqft)+Roof"


fig = go.Figure([
go.Bar(x = model_names, y = training_error, name="Training RMSE"),
go.Bar(x = model_names, y = validation_error, name="Validation RMSE"),
go.Bar(x = model_names, y = cv_error, name="Cross Val RMSE")
])

fig.update_yaxes(range=[180000,260000], title="RMSE")

fig


In [ ]:
# Plot 2 side-by-side residual plots for validation data

fig, ax = plt.subplots(1,2, figsize=(15, 5))


x_plt1 = linear_model_m3.predict(X_valid_m3)
y_plt1 = y_valid_m3 - linear_model_m3.predict(X_valid_m3)

x_plt2 = y_valid_m3
y_plt2 = y_valid_m3 - linear_model_m3.predict(X_valid_m3)


ax[0].scatter(x_plt1, y_plt1, alpha=.25)
ax[0].axhline(0, c='black', linewidth=1)
ax[0].set_xlabel(r'Predicted Log(Sale Price)')
ax[0].set_ylabel(r'Residuals: Log(Sale Price) - Predicted Log(Sale Price)');
ax[0].set_title("Model 3 Val Data: Residuals vs. Predicted Log(Sale Price)")

ax[1].scatter(x_plt2, y_plt2, alpha=.25)
ax[1].axhline(0, c='black', linewidth=1)
ax[1].set_xlabel(r'Log(Sale Price)')
ax[1].set_ylabel(r'Residuals: Log(Sale Price) - Predicted Log(Sale Price)');
ax[1].set_title("Model 3 Val Data: Residuals vs. Log(Sale Price)")



# Final Improved Model

In [ ]:
tr['Log Sale Price'] = np.log(tr['Sale Price'])
val['Log Sale Price'] = np.log(val['Sale Price'])
# print(tr)
def ohe_column(data, column_name):
    oh_enc = OneHotEncoder() 
    oh_enc.fit(data[[column_name]])
    
    dummies = pd.DataFrame(oh_enc.transform(data[[column_name]]).todense(),
                           columns=oh_enc.get_feature_names_out(),
                           index=data.index)
    
    return data.join(dummies)

def process_data_ec(df, dataset_type):

    data = df.copy()
    data['Log Building Square Feet'] = np.log(data['Building Square Feet'])
    data['Log Land Square Feet'] = np.log(data['Land Square Feet'])
    data['Sqrt Age'] = np.sqrt(data['Age'])   

    data = substitute_roof_material(data)
    
    if dataset_type == 3:
        data = data[['Log Building Square Feet', 'Log Land Square Feet', 'Sqrt Age', 
            'Roof Material', 'Basement', 'Basement Finish', 'Attic Type', 'Attic Finish',
            'Garage 1 Size', 'Garage Indicator', 'Construction Quality', 
            'Central Heating', 'Other Heating', 'Central Air', 'Site Desirability', 
            'Fireplaces', 'Floodplain', 'O\'Hare Noise',  
            'Road Proximity', 'Property Class', 'Sale Quarter', 'Garage 1 Material',
            'Garage 1 Attachment', 'Garage 1 Area','Garage 2 Attachment', 
            'Garage 2 Area', 'Design Plan', 'Town Code', 'Cathedral Ceiling']]
    else:
        data = data[['Log Sale Price', 'Log Building Square Feet', 'Log Land Square Feet', 
            'Sqrt Age', 'Roof Material', 'Basement', 'Basement Finish', 'Attic Type', 
            'Attic Finish', 'Garage 1 Size', 'Garage Indicator', 'Construction Quality',
            'Central Heating', 'Other Heating', 'Central Air', 'Site Desirability',
            'Fireplaces', 'Floodplain', 'O\'Hare Noise', 'Road Proximity', 'Property Class',
            'Sale Quarter', 'Garage 1 Material', 'Garage 1 Attachment', 'Garage 1 Area',
            'Garage 2 Attachment', 'Garage 2 Area', 'Design Plan', 'Town Code',
            'Cathedral Ceiling']]
    
    data = ohe_column(data, 'Basement')
    data = ohe_column(data, 'Basement Finish')
    data = ohe_column(data, 'Attic Type')
    data = ohe_column(data, 'Attic Finish')
    data = ohe_column(data, 'Garage 1 Size')
    data = ohe_column(data, 'Garage Indicator')
    data = ohe_column(data, 'Central Heating')
    data = ohe_column(data, 'Other Heating')
    data = ohe_column(data, 'Central Air')
    data = ohe_column(data, 'Design Plan')
    data = ohe_column(data, 'Cathedral Ceiling')
    data = ohe_column(data, 'Construction Quality')
    data = ohe_column(data, 'Site Desirability')
    data = ohe_column(data, 'Garage 1 Material')
    data = ohe_column(data, 'Garage 1 Attachment')
    data = ohe_column(data, 'Garage 1 Area')
    data = ohe_column(data, 'Garage 2 Attachment')
    data = ohe_column(data, 'Garage 2 Area')
    data = ohe_column(data, 'Fireplaces')
    data = ohe_column(data, 'Floodplain')
    data = ohe_column(data, 'O\'Hare Noise')
    data = ohe_column(data, 'Road Proximity')
    data = ohe_column(data, 'Property Class')
    data = ohe_column(data, 'Sale Quarter')
    data = ohe_column(data, 'Town Code')
    
    data = ohe_roof_material(data) 

    
    data = data.drop(columns=['Roof Material', 'Basement', 'Basement Finish', 'Attic Type', 
            'Attic Finish', 'Garage 1 Size', 'Garage Indicator', 'Construction Quality',
            'Central Heating', 'Other Heating', 'Central Air', 'Site Desirability',
             'Fireplaces', 'Floodplain', 'O\'Hare Noise', 'Road Proximity', 'Property Class', 
            'Sale Quarter', 'Garage 1 Material', 'Garage 1 Attachment', 'Garage 1 Area',
            'Garage 2 Attachment', 'Garage 2 Area', 'Design Plan', 'Town Code', 
                'Cathedral Ceiling'])
    
    return data

processed_train_ec = process_data_ec(tr, dataset_type=1)

processed_val_ec = process_data_ec(val, dataset_type=2)


X_train_ec = processed_train_ec.drop(columns='Log Sale Price')
y_train_ec = processed_train_ec['Log Sale Price']

X_valid_ec = processed_val_ec.drop(columns='Log Sale Price')
y_valid_ec = processed_val_ec['Log Sale Price']


# Take a look at the result
display(X_train_ec.head())
display(y_train_ec.head())

display(X_valid_ec.head())
display(y_valid_ec.head())

In [ ]:
# Create a Multiple Linear Regression Model

linear_model_ec = lm.LinearRegression()
linear_model_ec.fit(X_train_ec, y_train_ec)

y_predict_train_ec = linear_model_ec.predict(X_train_ec)

y_predict_valid_ec = linear_model_ec.predict(X_valid_ec)




In [ ]:
# Evaluate the RMSE for your model


# Training and test errors for the model 

y_predict_train_ec_original = np.exp(y_predict_train_ec)
y_predict_valid_ec_original = np.exp(y_predict_valid_ec)

training_error_ec = np.sqrt(mean_squared_error(np.exp(y_train_ec), y_predict_train_ec_original))
validation_error_ec = np.sqrt(mean_squared_error(np.exp(y_valid_ec), y_predict_valid_ec_original))


(print("Final Modal \nTraining RMSE:$ {}\nValidation RMSE:$ {}\n"
       .format(training_error_ec, validation_error_ec))
)


In [ ]:
fig = go.Figure([
go.Bar(x = ["Final Model"], y = [training_error_ec], name="Training RMSE"),
go.Bar(x = ["Final Model"], y = [validation_error_ec], name="Validation RMSE"),

])


fig
fig.update_yaxes(range=[140000,260000], title="RMSE")


In [ ]:
# Plot 2 side-by-side residual plots for validation data

fig, ax = plt.subplots(1,2, figsize=(15, 5))


x_plt1 = y_predict_valid_ec
y_plt1 = y_valid_ec - y_predict_valid_ec

x_plt2 = y_valid_ec
y_plt2 = y_valid_ec - y_predict_valid_ec


ax[0].scatter(x_plt1, y_plt1, alpha=.25)
ax[0].axhline(0, c='black', linewidth=1)
ax[0].set_xlabel(r'Predicted Log(Sale Price)')
ax[0].set_ylabel(r'Residuals: Log(Sale Price) - Predicted Log(Sale Price)');
ax[0].set_title("EC Val Data: Residuals vs. Predicted Log(Sale Price)")

ax[1].scatter(x_plt2, y_plt2, alpha=.25)
ax[1].axhline(0, c='black', linewidth=1)
ax[1].set_xlabel(r'Log(Sale Price)')
ax[1].set_ylabel(r'Residuals: Log(Sale Price) - Predicted Log(Sale Price)');
ax[1].set_title("EC Val Data: Residuals vs. Log(Sale Price)")
